In [1]:
import pandas as pd

In [2]:
ruta = r"C:\Users\Yudith\Desktop\Análisis de Datos\DATA ANALYTICS\ventas_supermercado_argentina_2024_2025.csv"

df = pd.read_csv(ruta)
df.head()

,id_pedido,id_cliente,fecha,hora,id_departamento,id_seccion,id_producto,nombre_producto,precio_unitario,cantidad,precio_total
0,347621,5824,2024-05-20,11,1,2,1002,Manzana Roja,2309.03,5,11545.15
1,202790,13674,2025-01-17,8,1,2,1004,Leche Entera,1831.60,1,1831.60
2,327127,7257,2025-04-23,15,3,3,1001,Banana,1906.34,2,3812.68
3,290785,9574,2025-03-21,11,2,1,1002,Manzana Roja,2041.93,3,6125.79
4,270165,13891,2025-01-23,19,2,5,1002,Manzana Roja,2575.80,4,10303.20


In [3]:
df_departamentos = df[['id_departamento']].drop_duplicates().reset_index(drop=True)
df_departamentos


,id_departamento
0,1
1,3
2,2


In [4]:
df_departamentos.to_csv('departamentos.csv', index=False)



In [5]:
ventas_por_departamento = df.groupby('id_departamento')['precio_total'].sum().reset_index()
ventas_por_departamento


,id_departamento,precio_total
0,1,26273364.71
1,2,27184888.31
2,3,25612669.00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_pedido        6000 non-null   int64  
 1   id_cliente       6000 non-null   int64  
 2   fecha            6000 non-null   object 
 3   hora             6000 non-null   int64  
 4   id_departamento  6000 non-null   int64  
 5   id_seccion       6000 non-null   int64  
 6   id_producto      6000 non-null   int64  
 7   nombre_producto  6000 non-null   object 
 8   precio_unitario  6000 non-null   float64
 9   cantidad         6000 non-null   int64  
 10  precio_total     6000 non-null   float64
dtypes: float64(2), int64(7), object(2)
memory usage: 515.8+ KB


LA FECHA SALE COMO OBJECT (TEXTO) HAY QUE PASARLO A NÚMERICA. EN ESTE ESTUDIO NO ES NECESARIO PORQUE NO LA VAMOS A USAR, PERO EN ANÁLISIS DE VENTAS SI DEBEMOS TENERLO EN CUENTA 

PARA CONVERTIR LA FECHA DE TEXTO A NÚMERO

In [7]:
df['fecha'] = pd.to_datetime(df['fecha'])


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_pedido        6000 non-null   int64         
 1   id_cliente       6000 non-null   int64         
 2   fecha            6000 non-null   datetime64[ns]
 3   hora             6000 non-null   int64         
 4   id_departamento  6000 non-null   int64         
 5   id_seccion       6000 non-null   int64         
 6   id_producto      6000 non-null   int64         
 7   nombre_producto  6000 non-null   object        
 8   precio_unitario  6000 non-null   float64       
 9   cantidad         6000 non-null   int64         
 10  precio_total     6000 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(7), object(1)
memory usage: 515.8+ KB


In [9]:
df.columns


Index(['id_pedido', 'id_cliente', 'fecha', 'hora', 'id_departamento',
       'id_seccion', 'id_producto', 'nombre_producto', 'precio_unitario',
       'cantidad', 'precio_total'],
      dtype='object')

Crear la canasta

In [10]:
df_cesta = df[['id_pedido', 'nombre_producto']]
df_cesta.head()


,id_pedido,nombre_producto
0,347621,Manzana Roja
1,202790,Leche Entera
2,327127,Banana
3,290785,Manzana Roja
4,270165,Manzana Roja


Agrupar productos por pedido

In [11]:
df_agrupado = (
    df_cesta
    .groupby('id_pedido')['nombre_producto']
    .apply(lambda x: ','.join(x))
)


Crear la matriz transaccional (0/1)

In [12]:
df_transacciones = df_agrupado.str.get_dummies(sep=',')
df_transacciones.head()


,Aceite Girasol,Arroz 1kg,Banana,Carne Picada,Fideos,Leche Entera,Manzana Roja,Pan Lactal,Pollo Entero,Yerba Mate 1kg
id_pedido,,,,,,,,,,
180002,0,0,0,0,0,0,1,0,0,0
180024,0,0,1,0,0,0,0,0,0,0
180034,0,0,0,0,1,0,0,0,0,0
180044,0,0,0,1,0,0,0,0,0,0
180051,0,0,0,0,0,0,0,1,0,0


Filtrar pedidos con más de 1 producto

In [13]:
df_agrupado = (
    df_cesta
    .groupby('id_pedido')['nombre_producto']
    .apply(lambda x: ','.join(x))
)

df_agrupado.head()


id_pedido
180002    Manzana Roja
180024          Banana
180034          Fideos
180044    Carne Picada
180051      Pan Lactal
Name: nombre_producto, dtype: object

Volver a crear la canasta

In [14]:
df_cesta = df_filtrado[['id_pedido', 'nombre_producto']]

df_agrupado = (
    df_cesta
    .groupby('id_pedido')['nombre_producto']
    .apply(lambda x: ','.join(x))
)

df_transacciones = df_agrupado.str.get_dummies(sep=',')


NameError: name 'df_filtrado' is not defined

Definir las métricas
🔸 Confianza
Si compro A, ¿qué probabilidad hay de comprar B?

In [ ]:
def confianza(antecedente, consecuente):
    compras_juntas = (
        (df_transacciones[antecedente] == 1) &
        (df_transacciones[consecuente] == 1)
    ).sum()
    
    compras_antecedente = (df_transacciones[antecedente] == 1).sum()
    
    return compras_juntas / compras_antecedente


Lift

In [ ]:
def lift(antecedente, consecuente):
    soporte_a = df_transacciones[antecedente].mean()
    soporte_b = df_transacciones[consecuente].mean()
    
    soporte_ab = (
        (df_transacciones[antecedente] == 1) &
        (df_transacciones[consecuente] == 1)
    ).mean()
    
    return soporte_ab / (soporte_a * soporte_b)


In [ ]:
confianza('Manzana Roja', 'Leche Entera')
lift('Manzana Roja', 'Leche Entera')


In [ ]:
# Soporte (%) de cada producto
soporte = df_transacciones.mean() * 100
soporte.sort_values(ascending=False)


FUNCIÓN CONFIANZA


In [ ]:
def confianza(antecedente, consecuente):
    conjunto_ac = df_transacciones[
        (df_transacciones[antecedente] == 1) &
        (df_transacciones[consecuente] == 1)
    ]
    return len(conjunto_ac) / df_transacciones[antecedente].sum()


FUNCIÓN LIFT



In [ ]:
def lift(antecedente, consecuente):
    soporte_a = df_transacciones[antecedente].mean()
    soporte_c = df_transacciones[consecuente].mean()

    conteo_ac = len(
        df_transacciones[
            (df_transacciones[antecedente] == 1) &
            (df_transacciones[consecuente] == 1)
        ]
    )

    soporte_ac = conteo_ac / len(df_transacciones)
    return soporte_ac / (soporte_a * soporte_c)


IMPORTAR COMBINACIONES

In [ ]:
from itertools import combinations


GENERAR REGLAS DE ASOCIACIÓN

In [ ]:
umbral_confianza = 0.02
asociaciones = []

for antecedente, consecuente in combinations(df_transacciones.columns, 2):

    soporte_a = df_transacciones[antecedente].mean()
    conf = confianza(antecedente, consecuente)

    if conf > umbral_confianza:
        asociaciones.append({
            'antecedente': antecedente,
            'consecuente': consecuente,
            'soporte_a': round(soporte_a * 100, 1),
            'confianza': round(conf * 100, 1),
            'lift': round(lift(antecedente, consecuente), 2)
        })


CREAR TABLA DE REGLAS

In [ ]:
df_asociaciones = pd.DataFrame(asociaciones)
df_asociaciones.sort_values(by='lift', ascending=False, inplace=True)

df_asociaciones


eL PRODUCTO QUE MAS SE COMPRA ES ACEITE DE GIRASOL(ANTECEDENTE) Y LA PROBABILIDAD QUE SE COMPRE JUNTO AL ACEITE ES LA CARNE PICADA (CONSECUENTE) DATO IMPORTANTE POR LO QUE HAY QUE PONERLOS JUNTOS PARA QUE SE VENDAN

SE TOMAN LAS REGLAS CUYO LIFT ES MAYOR A 1 PORQUE NOS DA LA PROBALIDAD DE QUE EL CLIENTE COMPRE EL PRODUCTO ANTECEDENTE JUNTO CON EL COCECUENTE. EJEMPLO ACEITE GIRASOL Y CARNE PICADA TIENEN UN LIFT MAYOR A 1

TABLA DE PRODUCTOS ÚNICOS

In [ ]:
productos_unicos = df[
    ['id_producto', 'id_seccion', 'id_departamento', 'nombre_producto']
].drop_duplicates()


ENRIQUECER LAS REGLAS

In [ ]:
df_asociaciones_enriquecido = df_asociaciones.merge(
    productos_unicos,
    left_on='antecedente',
    right_on='nombre_producto',
    how='left'
).drop(columns=['nombre_producto'])

df_asociaciones_enriquecido.columns = [
    'antecedente', 'consecuente', 'soporte_a',
    'confianza', 'lift',
    'id_producto_a', 'id_seccion_a', 'id_departamento_a'
]

df_asociaciones_enriquecido


El análisis de canasta de mercado permitió identificar asociaciones relevantes entre productos, destacándose combinaciones como Aceite de Girasol y Carne Picada, que presentan un lift superior a 1, indicando una relación de compra no aleatoria. Sin embargo, se observa que la mayoría de los pedidos contienen pocos productos, lo cual limita la cantidad de reglas significativas obtenidas.

“Cada fila muestra una regla de compra. El lift me indica si la relación es real. Valores mayores a 1 significan que los productos se compran juntos más de lo esperado. En nuestro caso, Aceite y Carne muestran una asociación relevante, mientras que otros productos frecuentes no necesariamente se compran juntos.”

EXPORTAR A CSV

In [ ]:
df_asociaciones_enriquecido.to_csv(
    'reglas.csv',
    index=False,
    sep=';',
    decimal=','
)
